In [44]:
# Connect to local DB
import os
import mysql.connector
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = "127.0.0.1"
db_name = "coursera_db"
engine = mysql.connector.connect(user=db_user, password=db_password, host=db_host, database=db_name)

# Run simple command to test connection
cur = engine.cursor()
cur.execute("SELECT CURDATE()")
row = cur.fetchone()
print("Current Date is: {0}".format(row[0]))

Current Date is: 2025-03-18


In [4]:
# Add a new column to pet_sale
cur.execute("ALTER TABLE pet_sale ADD COLUMN quantity INTEGER")

In [ ]:
import random 

# Add some values for the new column
cur.execute("SELECT id FROM pet_sale")
sale_ids = cur.fetchall()

for sale in sale_ids:
    rand_quantity = random.randint(1, 50)
    cur.execute(f"UPDATE pet_sale SET quantity = {rand_quantity} WHERE id = {sale[0]};")
    print(f"UPDATE pet_sale SET quantity = {rand_quantity} WHERE id = {sale[0]}")
    print(cur.rowcount)
    #print(sale[0])

In [ ]:
import pandas as pd
#Populate Medical DB Tables
# Note: Pandas read a csv, it treat first row as headers by default
# override that by adding 'header=None' in the read_csv method
med_departments = pd.read_csv("data\medical-departments.csv", header=None)
med_departments.columns = ["dept_id", "dept_name", "manager_id", "location_id"]
med_departments["manager_id"] = None
print(med_departments.head())

for i, department in med_departments.iterrows():
    sql = "INSERT INTO medical_departments VALUES (%s, %s, %s, %s)"
    cur.execute(sql, tuple(department))
    print("Record inserted")

In [ ]:
# Pull DB data to verify
cur.execute("SELECT * FROM medical_departments")
med_depart_data = cur.fetchall()

print(med_depart_data)

### `.iterrows()` in Pandas
The `.iterrows()` method in Pandas iterates over DataFrame rows as (index, Series) pairs. It allows you to loop through each row <b>one at a time</b>, but it is <b>slow for large DataFrames</b> since it operates row-wise.

### Basic Syntax
```
for index, row in df.iterrows():
    print(index)    # row index
    print(row)      # row data as a panda series

In [ ]:
med_history = pd.read_csv("data\medical-history.csv", header=None)
print(med_history.head())

for i, history in med_history.iterrows():
    sql = "INSERT INTO medical_history VALUES (%s, %s, %s, %s, %s, %s)"
    cur.execute(sql, tuple(history))
    print("Record inserted")

In [ ]:
med_locations = pd.read_csv("data\medical-locations.csv", header=None)
print(med_locations.head())

for i, location in med_locations.iterrows():
    sql = "INSERT INTO medical_locations VALUES (%s, %s, %s)"
    cur.execute(sql, tuple(location))
    print("Record inserted")

In [ ]:
med_procedures = pd.read_csv("data\medical-procedures.csv", header=None)
print(med_procedures.head())

for i, procedures in med_procedures.iterrows():
    sql = "INSERT INTO medical_procedures VALUES (%s, %s, %s, %s, %s)"
    cur.execute(sql, tuple(procedures))
    print("Record inserted")

In [ ]:
patients = pd.read_csv("data\patients.csv", header=None)
print(patients.head())

for i, patient in patients.iterrows():
    sql = "INSERT INTO patients VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, tuple(patient))
    print("Record inserted")

### What does `.commit` do?
`.commit()` is the method is database connection objects that finalizes changes made in a transcation and makes them permanent in the database.
<b>Without `.commit()`, the changes remain temporary and are lost when the connection is closed.</b>

### Why is `.commit` necessary?
When you modify data in MySQL, it does not immediately write those changes permanently. Instead, <b>it stores them in a temporary state (transction buffer)</b> until either:
- `.commit()` is called -> <b>makes the changes permanent</b>
- `.rollback()` is called -> <b>reverts changes</b>
- The connection closes <b>without committing</b> -> <b>changes are lost</b>

### Why can you still query uncommitted data?
Even if you havent committed, you can still <b>see uncommitted changes within the same connection</b> because:
- <b>MySQL supports transactions</b>, meaning the changes are visible only within the active session.
- Myour changes exist in a <b>temporary transaction buffer</b>, not yet written permanently to disk.

### How `.commit()` works internally?
When you call `.commit()` MySQL:
1. <b>Writes the transaction log</b> -> ensures durability.
2. <b>Releases locks</b> -> Allows other users to see the changes.
3. <b>Flushes changes to disk</b> -> Now permanently stored in the database.

In [53]:
engine.commit()

In [ ]:

cur.close()
engine.close()